## Make a static maps

### Download libraries and dataset

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import os
import numpy as np

from matplotlib.colors import Normalize,LogNorm
#######################################################

filename = '../data/cleaned/cleaned_dataset_v3.csv'

#shp_name = '../shapefiles/Belgium-4-Digit-Postcodes-2020.shp'
shp_name = '../shapefiles/belgium_map_simplified.shp'

#results_dir = './'
results_dir = '../map_visualization_results/'

#######################################################

df = pd.read_csv(filename,delimiter=',')

col = 'url'  # Drop the column Url because it needn't for analysis
if col in df.columns:
    df.drop(columns=[col], inplace=True)

## Create cdata for maps

In [2]:
### Add municipalities names
csv_name = '../shapefiles/postal_codes.csv'
df_post = pd.read_csv(csv_name, sep=';' )

df_post['postal_code'] = df_post['postal_code'].astype(int)

print( df_post.head(4) )


   postal_code    mun_name
0         1000     Brussel
1         1020     Brussel
2         1030  Schaarbeek
3         1040   Etterbeek


In [3]:
df_count = df.groupby('postal_code').size().reset_index(name='count')

percent_luxurious = (
    df.groupby('postal_code')['is_luxurious']
    .mean()  # mean - 
    .mul(100)  # convert to percents
    .astype(int)
    .reset_index(name='percent_luxurious')
)

price_variance = df.groupby('postal_code')['price'].var().reset_index(name='price_variance')

median_area = df.groupby('postal_code')['area'].median().reset_index(name='median_area')

median_price = df.groupby('postal_code')['price'].median().reset_index(name='median_price')
mean_price = df.groupby('postal_code')['price'].mean().reset_index(name='mean_price')

df['price_per_m2'] = df['price'] / df['area']

df_price_per_m2 = df.groupby('postal_code', as_index=False)['price_per_m2'].median()
df_price_per_m2.rename(columns={'price_per_m2': 'median_price_per_m2'}, inplace=True)

df_price_per_m2_average = df.groupby('postal_code', as_index=False)['price_per_m2'].mean()
df_price_per_m2_average.rename(columns={'price_per_m2': 'mean_price_per_m2'}, inplace=True)


######################################################################################

df_summary = pd.merge(df_count, percent_luxurious, on='postal_code', how='left')
df_summary = df_summary.merge(price_variance, on='postal_code', how='left')
df_summary = df_summary.merge(median_area, on='postal_code', how='left')
df_summary = df_summary.merge(median_price, on='postal_code', how='left')
df_summary = df_summary.merge(mean_price, on='postal_code', how='left')
df_summary = df_summary.merge(df_price_per_m2, on='postal_code', how='left')
df_summary = df_summary.merge(df_price_per_m2_average, on='postal_code', how='left')

df_summary = df_summary.merge(df_post[['postal_code', 'mun_name']], on='postal_code', how='left')


## prepare shapefile, join it with data

In [4]:
gdf = gpd.read_file(shp_name)

gdf['nouveau_PO'] = pd.to_numeric(gdf['nouveau_PO'], errors='coerce').astype('Int64')
df_summary['postal_code'] = pd.to_numeric(df_summary['postal_code'], errors='coerce').astype('Int64')

gdf = gdf.merge(df_summary, left_on='nouveau_PO', right_on='postal_code', how='left')

print( gdf.dtypes )

Join_Count                int32
nouveau_PO                Int64
FREQUENCY                 int32
CP_speciau                int32
Shape_Leng              float64
Shape_Area              float64
geometry               geometry
postal_code               Int64
count                   float64
percent_luxurious       float64
price_variance          float64
median_area             float64
median_price            float64
mean_price              float64
median_price_per_m2     float64
mean_price_per_m2       float64
mun_name                 object
dtype: object


## Load some specific libraries

In [5]:
import folium
import geopandas as gpd
import branca.colormap as cm
from folium.features import GeoJsonTooltip

### Create interactive map

In [6]:
tooltip_fields = [
    'mun_name',
    'postal_code',
    'count',
    'percent_luxurious',
    'median_area',
    'median_price',
    'mean_price',
    'median_price_per_m2',
    'mean_price_per_m2'
]

tooltip_aliases = [
    'Municipality:',
    'Postal code',
    'Number of properties',
    'Luxurious, %',
    'Median Area (sq.m.)',
    'Median Price (€)',
    'Average Price (€)',
    'Price per m2 (€)',
    'Average Price/m2'
]


In [7]:
# Function for adding layer by a variable
def add_variable_layer(gdf, variable, colormap, layer_name,show=False):
    layer = folium.FeatureGroup(name=layer_name,show=show)

    common_tooltip = GeoJsonTooltip(
        fields=tooltip_fields,
        aliases=tooltip_aliases,
        localize=True,
        sticky=True
    )
    
    folium.GeoJson(
        gdf,
        style_function=lambda feature: {
            'fillColor': colormap(feature['properties'][variable])
            if feature['properties'][variable] is not None else 'transparent',
            'color': 'black',
            'weight': 0.5,
            'fillOpacity': 0.7
        },
        tooltip=common_tooltip
    ).add_to(layer)
    layer.add_to(fmap)
    return layer

In [8]:
print(gdf.dtypes)

Join_Count                int32
nouveau_PO                Int64
FREQUENCY                 int32
CP_speciau                int32
Shape_Leng              float64
Shape_Area              float64
geometry               geometry
postal_code               Int64
count                   float64
percent_luxurious       float64
price_variance          float64
median_area             float64
median_price            float64
mean_price              float64
median_price_per_m2     float64
mean_price_per_m2       float64
mun_name                 object
dtype: object


In [9]:
# Create folium map
fmap = folium.Map(location=[50.85, 4.35], zoom_start=8, tiles=None)

# Create scales

##################################################################
##################################################################
# 'Blues_09' — светло-голубой → тёмно-синий
# 'Greens_09' — светло-зелёный → тёмно-зелёный
# 'Reds_09' — розовый → бордовый
# 'OrRd_09' — оранжево-красный
# 'PuBu_09' — пурпурно-синий
# 'BuGn_09' — голубой → зелёный
# 'YlGnBu_09' — жёлто-зелёно-синий
# 'YlOrBr_09' — жёлто-оранжево-коричневый
##################################################################

colormap_count = cm.linear.Blues_09.scale(0,350)
colormap_count.caption = 'Number of properties'
add_variable_layer(gdf, 'count', colormap_count, 'Number of properties')
colormap_count.add_to(fmap)

colormap_luxurious = cm.linear.Reds_09.scale(0,100)
colormap_luxurious.caption = 'Percent of luxurious Prop.'
add_variable_layer(gdf, 'percent_luxurious', colormap_luxurious, 'Percent of luxurious Prop.')
colormap_luxurious.add_to(fmap)

colormap_area = cm.linear.Reds_09.scale(40,300)
colormap_area.caption = 'Median Area (sq.m.)'
add_variable_layer(gdf, 'median_area', colormap_area, 'Median Area (sq.m.)')
colormap_area.add_to(fmap)

colormap_price = cm.linear.PuBu_09.scale(100,600_000)
colormap_price.caption = 'Median Price (€)'
add_variable_layer(gdf, 'median_price', colormap_price, 'Median Price (€)',True)
colormap_price.add_to(fmap)

colormap_price = cm.linear.PuBu_09.scale(100,600_000)
colormap_price.caption = 'Average Price (€)'
add_variable_layer(gdf, 'mean_price', colormap_price, 'Average Price (€)')
colormap_price.add_to(fmap)

colormap_per_m2 = cm.linear.Greens_09.scale(500,5000)
colormap_per_m2.caption = 'Price per m2 (€)'
add_variable_layer(gdf, 'median_price_per_m2', colormap_per_m2, 'Price per m?')
colormap_per_m2.add_to(fmap)

colormap_per_m2 = cm.linear.Greens_09.scale(500,5000)
colormap_per_m2.caption = 'Average Price / m2 (€)'
add_variable_layer(gdf, 'mean_price_per_m2', colormap_per_m2, 'Price per m2')
colormap_per_m2.add_to(fmap)


if(0):
    # === Логарифмическая шкала для дисперсии ===
    price_variance = gdf['price_variance'].replace(0, np.nan).dropna()
    log_min, log_max = np.log10(price_variance.min()), np.log10(price_variance.max())
    
    # Создаём непрерывную логарифмическую шкалу вручную
    def log_colormap(value):
        if value is None or value <= 0:
            return '#ffffff'
        # нормализуем в лог-пространстве
        scaled = (np.log10(value) - log_min) / (log_max - log_min)
        return cm.linear.OrRd_09(scaled)
    
    colormap_variance = cm.LinearColormap(
        colors=['#fee8c8', '#fdbb84', '#e34a33'],
        vmin=price_variance.min(),
        vmax=price_variance.max()
    )
    colormap_variance.caption = 'Price Variance (log scale)'
    add_variable_layer(gdf, 'price_variance', log_colormap, 'Price Variance (log)')
    colormap_variance.add_to(fmap)
##################################################################
##################################################################

# Add layers controls
folium.LayerControl(collapsed=False).add_to(fmap)

#######################################################
# Save the map into HTML file

#outname = 'interactive_municipalitets_map.html'
outname = 'interactive_municipalitets_map_v3.html'

filepath = os.path.join(results_dir, outname)

fmap.save(filepath)

# Show the map in JupNotebook
#fmap